In [53]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv

In [54]:
load_dotenv()

True

In [55]:
# tool create
@tool
def multiply(a: int, b: int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a * b

In [56]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [57]:
multiply.name, multiply.description, multiply.args

('multiply',
 'Given 2 numbers a and b this tool returns their product',
 {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}})

In [58]:
# tool binding
llm = ChatOpenAI()

In [59]:
llm_with_tools = llm.bind_tools([multiply])

In [60]:
# Tool Calling
llm_with_tools.invoke('Hi, how are you?')

AIMessage(content="Hello! I'm here and ready to help. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 60, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Brd62Mv8L3RSCWj1YCRgrjchd2Rrt', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f0e186b8-b88c-41d2-956e-7ba111bcd948-0', usage_metadata={'input_tokens': 60, 'output_tokens': 18, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [96]:
query = HumanMessage('can you multiply 3 with 1000?')

In [97]:
message = [query]

In [98]:
message

[HumanMessage(content='can you multiply 3 with 1000?', additional_kwargs={}, response_metadata={})]

In [99]:
result = llm_with_tools.invoke(message)

In [100]:
# llm_with_tools.invoke('can you multiply 3 with 10?').tool_calls

In [101]:
# result = llm_with_tools.invoke('can you multiply 3 with 10?')

In [102]:
message.append(result)

In [103]:
message

[HumanMessage(content='can you multiply 3 with 1000?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zx323K9rfxoH3uHnAhA9Lfuj', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 64, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrdAUoSiJU7iDYzlqpC5sgDN1kRNs', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e6faaa7b-3c55-44ae-82ce-723f58bc6e0d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_zx323K9rfxoH3uHnAhA9Lfuj', 'type': 'tool_call'}], usage_metadata={'input_tok

In [104]:
print(result.tool_calls[0])

{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_zx323K9rfxoH3uHnAhA9Lfuj', 'type': 'tool_call'}


In [105]:
# tool execution
tool_result = multiply.invoke(result.tool_calls[0])

In [106]:
message.append(tool_result)

In [107]:
message

[HumanMessage(content='can you multiply 3 with 1000?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zx323K9rfxoH3uHnAhA9Lfuj', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 64, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BrdAUoSiJU7iDYzlqpC5sgDN1kRNs', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e6faaa7b-3c55-44ae-82ce-723f58bc6e0d-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_zx323K9rfxoH3uHnAhA9Lfuj', 'type': 'tool_call'}], usage_metadata={'input_tok

In [108]:
llm_with_tools.invoke(message).content

'The product of 3 and 1000 is 3000.'